In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import warnings ; warnings.filterwarnings('ignore')

from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler

In [2]:
train = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/train.csv").drop("~", axis = 1) #경로지정
test = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/test.csv").drop("No", axis = 1)
sample_sub = pd.read_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/DATASET/submission.csv")

In [5]:
def preprocessing(data):
    # Lung cancer TNM stage
    #T
    data.loc[(data['M1a'] == 1 ) | (data['M1b'] == 1) | (data['M1c'] == 1), 'M0' ] = 0 
    data.loc[(data['M1a'] == 0) & (data['M1b'] == 0) & (data['M1c'] == 0),'M0'] = 1

    #N
    data.loc[(data['N1'] == 1 ) | (data['N2'] == 1) | (data['N3'] == 1), 'N0'] = 0
    data.loc[(data['N1'] == 0) & (data['N2'] == 0) & (data['N3'] == 0), 'N0'] = 1
    #M
    
    #stage
    data['TNM_stage'] = ''

    stage_map = {

        ('T1', 'N0'): 'IA', ('T1', 'N1'): 'IIA', ('T1', 'N2'): 'IIB', ('T1', 'N3'): 'IIIA', 
        ('T1a', 'N0'): 'IA', ('T1a', 'N1'): 'IIA', ('T1a', 'N2'): 'IIB', ('T1a', 'N3'): 'IIIA',
        ('T1b', 'N0'): 'IA', ('T1b', 'N1'): 'IIA', ('T1b', 'N2'): 'IIB', ('T1b', 'N3'): 'IIIA', 
        ('T1c', 'N0'): 'IA', ('T1c', 'N1'): 'IIA', ('T1c', 'N2'): 'IIB', ('T1c', 'N3'): 'IIIA', 
        ('T2', 'N0'): 'IB', ('T2', 'N1'): 'IIB', ('T2', 'N2'): 'IIIA', ('T2', 'N3'): 'IIIB',
        ('T2a', 'N0'): 'IB', ('T2a', 'N1'): 'IIB', ('T2a', 'N2'): 'IIIA', ('T2a', 'N3'): 'IIIB', 
        ('T2b', 'N0'): 'IIA', ('T2b', 'N1'): 'IIB', ('T2b', 'N2'): 'IIIA', ('T2b', 'N3'): 'IIIB', 
        ('T3', 'N0'): 'IIB', ('T3', 'N1'): 'IIIA', ('T3', 'N2'): 'IIIA', ('T3', 'N3'): 'IIIB',
        ('T4', 'N0'): 'IIIA', ('T4', 'N1'): 'IIIA', ('T4', 'N2'): 'IIIB', ('T4', 'N3'): 'IIIC'
    }

    for t in ['T1', 'T1a', 'T1b', 'T1c', 'T2', 'T2a', 'T2b', 'T3', 'T4']:
        for n in ['N0', 'N1', 'N2', 'N3']:
            mask = (data[t] == 1) & (data[n] == 1)
            target = stage_map.get((t, n), 'Invalid TNM')
            data.loc[mask, 'TNM_stage'] = target
            if target == 'Invalid TNM':
                print((t, n))

    data.loc[data['M1a'] == 1, 'TNM_stage'] = 'IVA'
    data.loc[data['M1b'] == 1, 'TNM_stage'] = 'IVA'
    data.loc[data['M1c'] == 1, 'TNM_stage'] = 'IVB'

    stage_encode = {'' : 0, 'IA' : 1, "IB" : 2, "IIA" : 3, "IIB" : 4 , "IIIA" : 5, "IIIB" : 6, "IIIC": 7, "IVA": 8, "IVB":9}
    stage_encode2 = {0 : 0, 1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:3, 8:4, 9:4}
    data['TNM_stage'] = data['TNM_stage'].map(stage_encode)
    data['TNM_bigstage'] = data['TNM_stage'].map(stage_encode2)
    
    # Surgery
    data['Therapy_count'] = data['Operation'] + data['Chemotherapy'] + data['Radiation Therapy']

    # Other
    data['BMI'] = data['Weight'] / (data['Height'] / 100) ** 2

    # EGFR mutation Detection encoding 99 -> 0
    data['EGFR mutation Detection'] = data['EGFR mutation Detection'].replace(99, 2)

    # Type of Drink encoding 99 -> 0
    data['Type of Drink'] = data['Type of Drink'].replace(99, 0)

    # Mu
    return data

In [6]:
train = preprocessing(train)
test = preprocessing(test)

In [7]:
train_data = TabularDataset(train)
test_data = TabularDataset(test)

In [32]:
save_path = '/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/MODEL_SAVE/ag_EDA/ds_sub_fit/sub_fit_ho' #save 경로

predictor = TabularPredictor.load(path = save_path)

In [36]:
predictor.leaderboard().head(20)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_r22_BAG_L4,0.798155,accuracy,32.564257,24606.306014,0.036421,11.433725,4,True,391
1,LightGBM_r15_BAG_L4,0.797705,accuracy,32.554415,24611.079384,0.026579,16.207095,4,True,358
2,XGBoost_r49_BAG_L4,0.797705,accuracy,32.566827,24612.258777,0.038990,17.386488,4,True,378
3,CatBoost_r137_BAG_L4,0.797592,accuracy,32.551978,24607.364353,0.024141,12.492064,4,True,331
4,LightGBMLarge_BAG_L4,0.797592,accuracy,32.558640,24661.827042,0.030804,66.954753,4,True,321
5,LightGBM_r121_BAG_L4,0.797592,accuracy,32.621258,24766.213320,0.093422,171.341031,4,True,395
6,CatBoost_r5_BAG_L4,0.797480,accuracy,32.551011,24604.931155,0.023174,10.058866,4,True,379
7,LightGBM_r131_BAG_L4,0.797480,accuracy,32.566957,24630.288971,0.039120,35.416682,4,True,324
8,XGBoost_r95_BAG_L4,0.797367,accuracy,32.556289,24604.084521,0.028452,9.212232,4,True,414
9,LightGBM_r135_BAG_L4,0.797367,accuracy,32.581549,24657.665513,0.053712,62.793224,4,True,390


In [42]:
predictor.set_model_best(model = "XGBoost_r22_BAG_L4")

In [43]:
pred_proba = predictor.predict_proba(test_data)

In [47]:
pred_proba.describe()

,0,1
count,5000.000000,5000.000000
mean,0.775601,0.224399
std,0.028855,0.028855
min,0.646805,0.118393
25%,0.756745,0.205277
50%,0.775519,0.224481
75%,0.794723,0.243255
max,0.881607,0.353195


In [48]:
for i in np.arange(0.118393, 0.353195, 0.0001):
    print(i, sum(np.where(pred_proba.iloc[:, 1] > i, 1, 0)))


0.118393 4999
0.118493 4999
0.118593 4999
0.118693 4999
0.11879300000000001 4999
0.11889300000000001 4999
0.11899300000000002 4999
0.11909300000000002 4999
0.11919300000000002 4999
0.11929300000000002 4999
0.11939300000000003 4999
0.11949300000000003 4999
0.11959300000000003 4999
0.11969300000000004 4999
0.11979300000000004 4999
0.11989300000000004 4999
0.11999300000000004 4999
0.12009300000000005 4999
0.12019300000000005 4999
0.12029300000000005 4999
0.12039300000000006 4999
0.12049300000000006 4999
0.12059300000000006 4999
0.12069300000000006 4999
0.12079300000000007 4999
0.12089300000000007 4999
0.12099300000000007 4999
0.12109300000000008 4999
0.12119300000000008 4999
0.12129300000000008 4999
0.12139300000000008 4999
0.12149300000000009 4999
0.12159300000000009 4999
0.12169300000000009 4999
0.1217930000000001 4999
0.1218930000000001 4999
0.1219930000000001 4999
0.1220930000000001 4999
0.12219300000000011 4999
0.12229300000000011 4999
0.12239300000000011 4999
0.12249300000000012 499

In [51]:
pred = np.where(pred_proba.iloc[:, 1] > 0.305, 1, 0)

In [52]:
sample_sub['Death'] = pred
sample_sub["Death"].value_counts()

Death
0    4983
1      17
Name: count, dtype: int64

In [53]:
sample_sub.to_csv("/Users/mingu/Desktop/CODING/2024_1 Machine Learning/Kaggle Project 2/SUBMISSION/submission_5_14_2.csv", index = False) #경로지정